In [1]:


! python -m pip install -U sentence-transformers
# ! pip install sparqlwrapper
! python -m pip install langchain 
! python -m pip install InstructorEmbedding
! python -m pip install tiktoken
! python -m pip install faiss-gpu



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not curre

In [2]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader,Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from sentence_transformers import SentenceTransformer,util
import json
from langchain.document_loaders import CSVLoader,TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from tqdm.notebook import tqdm
embedding = SentenceTransformerEmbeddings(model_name="hkunlp/instructor-xl",model_kwargs={"device":"cuda"})

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [5]:
path="/kaggle/input/webis-clickbait-22/"

In [6]:

def final_creator(file):
    final_dict={'id':[],'postText':[],'targetTitle':[],'targetDescription':[],"exctractedParagraph":[]}
    data=pd.read_json(path+file,lines=True)
    progress=tqdm(range(len(data)))
    for i in progress:
        q=data.iloc[i]
        l=q['postText'][0]
        a=q['targetParagraphs']
        target_title=q['targetTitle']
        a="\n".join(a)
        with open("a.txt","w") as f:
            f.write(a)
        loader = TextLoader("a.txt")
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=100)
        texts = text_splitter.split_documents(documents)
        db_embeddings = FAISS.from_documents(texts,embedding)
        retriever = db_embeddings.as_retriever(search_kwargs={"k":2})
        top_triples = retriever.get_relevant_documents(l)
        context=""
        for i in top_triples:
            context+=i.page_content
        final_dict['id'].append(q['uuid'])
        final_dict['postText'].append(l)
        final_dict['targetTitle'].append(target_title)
        final_dict['targetDescription'].append(q['targetDescription'])
        final_dict['exctractedParagraph'].append(context)
    final_data=pd.DataFrame(final_dict)
    final_data.to_json("final_data_"+file+".json")

In [7]:
train_file="train.jsonl"
val_file="validation.jsonl"
test_file="test.jsonl"
# final_creator(train_file)
final_creator(val_file)
# final_creator(test_file)

  0%|          | 0/800 [00:00<?, ?it/s]